In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

num_qubits = 7
dataset_filename = "dataset/dataset_tesi/NN1_Dataset(<=10Cx)_balanced1.csv"
df = pd.read_csv(dataset_filename)

links = [set([0,1]), set([1,2]), set([1,3]), set([3,5]), set([4,5]), set([5,6])]
def generate_columns(header, links, in_links=False):
    if in_links:
        return [header+str(i)+str(j) for i in range(num_qubits) for j in range(num_qubits) if set([i,j]) in links]
    else:
        return [header+str(i)+str(j) for i in range(num_qubits) for j in range(num_qubits) if set([i,j]) not in links and i!=j]

useless_columns = ['Unnamed: 0', 'last_update_date', 'N_qubtis', 'N_measure', 'N_cx', 'backend_name']
useless_columns += generate_columns("cx_", links)
useless_columns += generate_columns("edge_length_", links)
useless_columns += generate_columns("edge_error_", links)
useless_columns += ["measure_"+str(i) for i in range(num_qubits)]
# Note that cx/edge_error/edge_length_xy is not neccessarily the same as cx/edge_length/edge_error_yx
df.drop(columns=useless_columns, inplace=True)
df.drop_duplicates(inplace=True)

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train, test = train_test_split(df, test_size=0.2)
df_train_x, df_train_y= train.iloc[:, :-num_qubits], train.iloc[:, -num_qubits:]
df_test_x, df_test_y= test.iloc[:, :-num_qubits], test.iloc[:, -num_qubits:]

train_x = scaler.fit_transform(df_train_x)
test_x = scaler.fit_transform(df_test_x)


# for every row in y, convert to 1 hot-encoding and flatten
train_y = []
for _, row in df_train_y.iterrows():
    train_y.append(pd.get_dummies(row).values.flatten())
train_y = np.array(train_y)

test_y = []
for _, row in df_test_y.iterrows():
    test_y.append(pd.get_dummies(row).values.flatten())
test_y = np.array(test_y)

(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

((7026, 57), (7026, 49), (1757, 57), (1757, 49))

# Simple MLP

In [3]:
import torch
from torch.nn import Linear
import torch.nn.functional as F


class MLP(torch.nn.Module):
    def __init__(self, num_features, num_classes, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin4 = Linear(hidden_channels, num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        x = self.lin3(x)
        x = x.relu()
        x = self.lin4(x)
        return x

/home/jacky/anaconda3/envs/eece571f/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = MLP(57, 49, hidden_channels=128)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

torch_train_x = torch.tensor(train_x, dtype=torch.float)
torch_train_y = torch.tensor(train_y, dtype=torch.float)
torch_test_x = torch.tensor(test_x, dtype=torch.float)
torch_test_y = torch.tensor(test_y, dtype=torch.float)

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(torch_train_x)  # Perform a single forward pass.
      loss = criterion(out, torch_train_y)  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(torch_train_x)
      pred = out  # Use the class with highest probability.

      test_correct = 0
      for i, j in zip(pred, torch_train_y):
          pred_i = np.argmax(i.detach().numpy().reshape(7,7), axis=1)
          train_y = np.argmax(j.detach().numpy().reshape(7,7), axis=1)
          test_correct += np.array_equal(pred_i, train_y)

      test_acc = int(test_correct) / int(torch_train_y.shape[0])  # Derive ratio of correct predictions.
      return test_acc


In [5]:
for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 27.0803
Epoch: 002, Loss: 25.3286
Epoch: 003, Loss: 21.0375
Epoch: 004, Loss: 18.8970
Epoch: 005, Loss: 18.7950
Epoch: 006, Loss: 18.8510
Epoch: 007, Loss: 17.6396
Epoch: 008, Loss: 17.4886
Epoch: 009, Loss: 17.6235
Epoch: 010, Loss: 17.5336
Epoch: 011, Loss: 17.3202
Epoch: 012, Loss: 17.0892
Epoch: 013, Loss: 16.8502
Epoch: 014, Loss: 16.7429
Epoch: 015, Loss: 16.7733
Epoch: 016, Loss: 16.7455
Epoch: 017, Loss: 16.6404
Epoch: 018, Loss: 16.5606
Epoch: 019, Loss: 16.5409
Epoch: 020, Loss: 16.5243
Epoch: 021, Loss: 16.4662
Epoch: 022, Loss: 16.3874
Epoch: 023, Loss: 16.3353
Epoch: 024, Loss: 16.3327
Epoch: 025, Loss: 16.3433
Epoch: 026, Loss: 16.3184
Epoch: 027, Loss: 16.2742
Epoch: 028, Loss: 16.2438
Epoch: 029, Loss: 16.2258
Epoch: 030, Loss: 16.2062
Epoch: 031, Loss: 16.1782
Epoch: 032, Loss: 16.1525
Epoch: 033, Loss: 16.1382
Epoch: 034, Loss: 16.1202
Epoch: 035, Loss: 16.0872
Epoch: 036, Loss: 16.0579
Epoch: 037, Loss: 16.0411
Epoch: 038, Loss: 16.0165
Epoch: 039, 

In [7]:
training_acc = test()
print(f'Training Accuracy: {training_acc:.4f}')

Training Accuracy: 0.9183


# Simple GNN

In [8]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(3, 2)
        self.conv2 = GCNConv(2, 2)
        self.conv3 = GCNConv(2, 1)
        self.classifier = Linear(1, 7)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(3, 2)
  (conv2): GCNConv(2, 2)
  (conv3): GCNConv(2, 1)
  (classifier): Linear(in_features=1, out_features=7, bias=True)
)


In [9]:
node_prefix = ["T1", "T2", "readout_error"]
node_features_x = []
for k in range(df_train_x.shape[0]):
    node_features_j = []
    for j in range(num_qubits):
        row_features = df_train_x.iloc[k][[i + "_" + str(j) for i in node_prefix]].values.flatten()
        node_features_j.append(row_features)
    node_features_j = scaler.fit_transform(node_features_j)
    node_features_x.append(node_features_j)
node_features_x = np.array(node_features_x)

edge_prefix = ["cx_", "edge_length_", "edge_error_"]
edge_index = [[],[]]
edge_features_x = []
for k in range(df_train_x.shape[0]):
    edge_features_j = []
    for i in range(num_qubits):
        for j in range(num_qubits):
            if set([i,j]) in links:
                row_features = df_train_x.iloc[k][[prefix + str(i) + str(j) for prefix in edge_prefix]].values.flatten()
                edge_features_j.append(row_features)
                if k == 0: # only need to do this once
                    edge_index[0].append(i)
                    edge_index[1].append(j)
    edge_features_j = scaler.fit_transform(edge_features_j)
    edge_features_x.append(edge_features_j)
edge_features_x = np.array(edge_features_x)
edge_index = np.array(edge_index)

node_labels = df_train_y.to_numpy()
print(node_features_x.shape, edge_index.shape, edge_features_x.shape, node_labels.shape)




(7026, 7, 3) (2, 12) (7026, 12, 3) (7026, 7)


In [10]:
torch_node_features_x = torch.tensor(node_features_x, dtype=torch.float)
torch_edge_index = torch.tensor(edge_index, dtype=torch.int)
torch_edge_features_x = torch.tensor(edge_features_x, dtype=torch.float)
torch_node_labels = torch.tensor(node_labels, dtype=torch.long)

In [11]:
model = GCN()
criterion = torch.nn.CrossEntropyLoss()  #Initialize the CrossEntropyLoss function.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Initialize the Adam optimizer.

def train(node_features, edge_index, node_labels):

    optimizer.zero_grad()  # Clear gradients.
    out, h = model(node_features, edge_index)  # Perform a single forward pass.
    loss = criterion(out, node_labels)  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss, h

for epoch in range(401):
    loss, h = train(torch_node_features_x[0], torch_edge_index, torch_node_labels[0])
    print(f'Epoch: {epoch}, Loss: {loss}')

Epoch: 0, Loss: 2.109177827835083
Epoch: 1, Loss: 2.0964386463165283
Epoch: 2, Loss: 2.0848934650421143
Epoch: 3, Loss: 2.074511766433716
Epoch: 4, Loss: 2.0652151107788086
Epoch: 5, Loss: 2.0568840503692627
Epoch: 6, Loss: 2.049377202987671
Epoch: 7, Loss: 2.042555809020996
Epoch: 8, Loss: 2.0363032817840576
Epoch: 9, Loss: 2.0305304527282715
Epoch: 10, Loss: 2.0251712799072266
Epoch: 11, Loss: 2.0201785564422607
Epoch: 12, Loss: 2.0155184268951416
Epoch: 13, Loss: 2.011164903640747
Epoch: 14, Loss: 2.0070948600769043
Epoch: 15, Loss: 2.003286361694336
Epoch: 16, Loss: 1.999714732170105
Epoch: 17, Loss: 1.9963533878326416
Epoch: 18, Loss: 1.993174433708191
Epoch: 19, Loss: 1.9901496171951294
Epoch: 20, Loss: 1.9872545003890991
Epoch: 21, Loss: 1.9844681024551392
Epoch: 22, Loss: 1.981775164604187
Epoch: 23, Loss: 1.9791641235351562
Epoch: 24, Loss: 1.9766274690628052
Epoch: 25, Loss: 1.9741599559783936
Epoch: 26, Loss: 1.9717581272125244
Epoch: 27, Loss: 1.9694193601608276
Epoch: 28, 